# Analyze This 2017 - Envible

Change directory

In [ ]:
setwd("~/coding/Envible-VITVellore-AMAX-Analyze-This-2017")

Read data

In [ ]:
data <- read.csv('Dataset/Training_Dataset.csv')

Drop 
- cm_key - Unique for every customer
- mvar1 - Same for all customers

In [ ]:
data <- data[, ! names(data) %in% c("mvar1"), drop = FALSE]
data <- data[, ! names(data) %in% c("cm_key"), drop = FALSE]

### Create binary and multi-class targets

Multi class - 

In [ ]:
data$target <- rep("None",nrow(data))
data$target[data$mvar49==1] <- "Supp"
data$target[data$mvar50==1] <- "Elite"
data$target[data$mvar51==1] <- "Credit"
length(data$target)

Binary - 

In [ ]:
data$targetBin <- rep(0,nrow(data))
data$targetBin[data$mvar49==1] <- 1
data$targetBin[data$mvar50==1] <- 1
data$targetBin[data$mvar51==1] <- 1
length(data$targetBin)

Make targets factors

In [ ]:
data$target <- as.factor(data$target)
data$targetBin <- as.factor(data$targetBin)

Drop columns used to determine target

In [ ]:
data <- data[, ! names(data) %in% c("mvar46"), drop = FALSE]
data <- data[, ! names(data) %in% c("mvar47"), drop = FALSE]
data <- data[, ! names(data) %in% c("mvar48"), drop = FALSE]
data <- data[, ! names(data) %in% c("mvar49"), drop = FALSE]
data <- data[, ! names(data) %in% c("mvar50"), drop = FALSE]
data <- data[, ! names(data) %in% c("mvar51"), drop = FALSE]

Push target columns to the end

In [ ]:
multi <- data$target 
data <- data[, ! names(data) %in% c("target"), drop = FALSE]
data['target'] <- multi
bin <- data$targetBin 
data <- data[, ! names(data) %in% c("targetBin"), drop = FALSE]
data['targetBin'] <- bin

In [ ]:
str(data)

Make factors

In [ ]:
data$mvar2 <- as.factor(data$mvar2)
data$mvar4 <- as.factor(data$mvar4)
data$mvar10 <- as.factor(data$mvar10)
data$mvar12 <- as.factor(data$mvar12)
data$mvar13 <- as.factor(data$mvar13)
data$mvar14 <- as.factor(data$mvar14)
data$mvar15 <- as.factor(data$mvar15)
data$mvar40 <- as.factor(data$mvar40)
data$mvar41 <- as.factor(data$mvar41)
data$mvar42 <- as.factor(data$mvar42)
data$mvar43 <- as.factor(data$mvar43)
data$mvar44 <- as.factor(data$mvar44)
data$mvar45 <- as.factor(data$mvar45)

Start H2O instance

In [ ]:
library(h2o)
h2o.init(nthreads=-1,max_mem_size="3g",enable_assertions=FALSE)

### Perform Binary Classification

Split dataset into training and testing

In [ ]:
set.seed(1)
indexes = sample(1:nrow(data),size=0.2*nrow(data))
test=data[indexes,]
train=data[-indexes,]

Create H2O variables

In [ ]:
train.hex<-as.h2o(train)
test.hex<-as.h2o(test)

Perform classification

In [ ]:
classification_model_binary<-h2o.gbm(
        training_frame =train.hex,
        validation_frame=test.hex,
        x=-c(45),
        y=46,
        seed =100,
        ntrees = 1000,
        learn_rate=0.01,
        stopping_rounds = 5, stopping_tolerance = 1e-4,
        stopping_metric = "AUC",
        sample_rate = 0.8,
        col_sample_rate = 0.8,
        score_tree_interval = 10
)

See model details

In [ ]:
print(classification_model_binary)

### Perform 3 class classification (Supp, Elite, Credit)

Remove training examples with None as card type

In [ ]:
card.data <- data[data$target != 'None',]

Split dataset into training and testing

In [ ]:
set.seed(1)
indexes = sample(1:nrow(card.data),size=0.2*nrow(card.data))
card.test=card.data[indexes,]
card.train=card.data[-indexes,]

In [ ]:
str(card.data)

Create H2O variables

In [ ]:
card.train.hex<-as.h2o(card.train)
card.test.hex<-as.h2o(card.test)

Perform classification

In [ ]:
regression_model_multi<-h2o.xgboost(
        training_frame =card.train.hex,
        validation_frame=card.test.hex,
        x=-c(46),
        y=45,
        nfolds=10,
        seed =100,
        ntrees = 1000,
        learn_rate=0.01,
        stopping_rounds = 5, stopping_tolerance = 1e-4,
        stopping_metric = "logloss", 
        sample_rate = 0.8,
        col_sample_rate = 0.8,
        score_tree_interval = 10
)

See model details

In [63]:
print(regression_model_multi)

Model Details:

H2OMultinomialModel: xgboost
Model ID:  XGBoost_model_R_1503833108862_7191 
Model Summary: 
  number_of_trees
1            1000


H2OMultinomialMetrics: xgboost
** Reported on training data. **
** Metrics reported on training frame **

Training Set Metrics: 

MSE: (Extract with `h2o.mse`) 0.1805814
RMSE: (Extract with `h2o.rmse`) 0.4249487
Logloss: (Extract with `h2o.logloss`) 0.5325742
Mean Per-Class Error: 0.1960355
Confusion Matrix: Extract with `h2o.confusionMatrix(<model>,train = TRUE)`)
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
          0    1    2  Error            Rate
0      1694  315  190 0.2296 =   505 / 2,199
1       131 1910   91 0.1041 =   222 / 2,132
2       271  434 2067 0.2543 =   705 / 2,772
Totals 2096 2659 2348 0.2016 = 1,432 / 7,103

Hit Ratio Table: Extract with `h2o.hit_ratio_table(<model>,train = TRUE)`
Top-3 Hit Ratios: 
  k hit_ratio
1 1  0.798395
2 2  0.964100
3 3  1.000000


H2OMultinomialMetrics: xgboost
** 

### Perform prediction on leaderboard dataset

Load dataset

In [ ]:
leader <- read.csv('Dataset/Leaderboard_Dataset.csv')

Save cm_key

In [ ]:
cm_key <- leader['cm_key']

Perform data manipulations as performed on training set

In [ ]:
leader <- leader[, ! names(leader) %in% c("mvar1"), drop = FALSE]
leader <- leader[, ! names(leader) %in% c("cm_key"), drop = FALSE]

In [ ]:
leader$mvar2 <- as.factor(leader$mvar2)
leader$mvar4 <- as.factor(leader$mvar4)
leader$mvar10 <- as.factor(leader$mvar10)
leader$mvar12 <- as.factor(leader$mvar12)
leader$mvar13 <- as.factor(leader$mvar13)
leader$mvar14 <- as.factor(leader$mvar14)
leader$mvar15 <- as.factor(leader$mvar15)
leader$mvar40 <- as.factor(leader$mvar40)
leader$mvar41 <- as.factor(leader$mvar41)
leader$mvar42 <- as.factor(leader$mvar42)
leader$mvar43 <- as.factor(leader$mvar43)
leader$mvar44 <- as.factor(leader$mvar44)
leader$mvar45 <- as.factor(leader$mvar45)

Create H2O variable

In [ ]:
leader.hex<-as.h2o(leader)

Perform binary prediction

In [ ]:
predictions_binary <- as.data.frame(h2o.predict(classification_model_binary, leader.hex))

In [ ]:
leader['predict'] <- predictions_binary['predict']
leader['p1'] <- predictions_binary['p1']
leader["cm_key"] <- cm_key

In [ ]:
dim(subset(leader, p1 > 0.3698))

In [ ]:
leader <- subset(leader, p1 > 0.3698)

In [ ]:
leader <- leader[with(leader, order(-p1)), ]

In [ ]:
cm_key_multi <- leader['cm_key']
leader <- leader[, ! names(leader) %in% c("cm_key"), drop = FALSE]
leader <- leader[, ! names(leader) %in% c("p1"), drop = FALSE]
leader <- leader[, ! names(leader) %in% c("predict"), drop = FALSE]

In [ ]:
dim(leader)

Perform 3-class classification

In [57]:
multi.leader.hex<-as.h2o(leader)

  |======================================================================| 100%


In [62]:
h2o.predict(classification_model_binary, multi.leader.hex)

  |======================================================================| 100%


  predict        p0        p1
1       1 0.2326839 0.7673161
2       1 0.2381883 0.7618117
3       1 0.2587879 0.7412121
4       1 0.2906512 0.7093488
5       1 0.2922629 0.7077371
6       1 0.2929470 0.7070530

[1000 rows x 3 columns] 